## Inicio - importação das bibliotecas

In [1]:
import pandas as pd
import re

### Abrindo base de dados

In [2]:
colunas = ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']

# LINHA DE SELECAO DO INPUT
df = pd.read_csv(
    "dataset_clinica20252.csv",
    sep="|",               
    encoding="utf-8"     
)

print(df.shape)
print(df.head(20))

(19800, 6)
      cd_causa             cd_atendimento                 ds_Acao_Judicial  \
0   CIB0500064  0825789-84.2025.8.18.0140  90 - ACAO DE REPARACAO DE DANOS   
1   CIB0505587  1004697-72.2025.8.26.0066  90 - ACAO DE REPARACAO DE DANOS   
2   CIB0508201  0800423-07.2025.8.15.0761  90 - ACAO DE REPARACAO DE DANOS   
3   CIB0514647  1004875-69.2025.8.26.0438  90 - ACAO DE REPARACAO DE DANOS   
4   CIB0500604  0010630-50.2025.8.27.2706  90 - ACAO DE REPARACAO DE DANOS   
5   CIB0511068  0092601-36.2025.8.05.0001  90 - ACAO DE REPARACAO DE DANOS   
6   CIB0515030  0801575-51.2025.8.18.0068  90 - ACAO DE REPARACAO DE DANOS   
7   CIB0515018  0801452-38.2025.8.18.0073  90 - ACAO DE REPARACAO DE DANOS   
8   CIB0515702  0800895-81.2025.8.10.0038  90 - ACAO DE REPARACAO DE DANOS   
9   CIB0514229  0148716-17.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
10  CIB0514230  0148734-38.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
11  CIB0524760  0161843-22.2025.8.04.1000  90 - ACAO 

### Etapa 1 - Função de captar valor pedido e número do CNPJ

In [4]:
def analisa_regex(pedido: str, qualificacao: str):
    """
    Analisa texto de pedido e qualificação para extrair valor da causa e CNPJ.
    Retorna o valor numérico do valor da causa, nível de confiança nesse valor numérico e CNPJ extraído.
    
    Args:
        pedido (str): Texto da coluna ds_Pedidos
        qualificacao (str): Texto da coluna ds_Qualificacao
        
    Returns:
        dict: {
            'valor_numerico': float ou None,
            'confianca': 'alta', 'média', 'baixa' ou None,
            'cnpj': str (14 dígitos) ou None
        }
    """
    
    # ========================================================================
    # EXTRAÇÃO DO VALOR DA CAUSA
    # ========================================================================
    
    valor_numerico = None
    confianca = None
    
    if pedido and isinstance(pedido, str) and pedido.strip() != '':
        padroes = [
            # Padrão 1: "Dá-se à causa[,] o valor [de] R$" (aceita vírgula após "causa")
            {
                'nome': 'dá-se à causa (com vírgula)',
                'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão|dando-se)\s+(?:à|a)\s+(?:presente\s+)?(?:causa|demanda|ação)\s*,?\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 2: "valor total da pretensão condenatória" / "ou seja, a quantia de"
            {
                'nome': 'valor da pretensão/quantia',
                'regex': r'(?:valor\s+(?:do\s+)?total\s+da\s+pretensão\s+condenatória|ou\s+seja,?\s+a\s+quantia)\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 3: "todos os efeitos de direito" (comum no final)
            {
                'nome': 'para efeitos de direito',
                'regex': r'para\s+(?:todos\s+os\s+)?efeitos\s+de\s+direito[\.,]?\s*R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 4: "Dá-se/dando-se à causa/demanda"
            {
                'nome': 'dá-se/dando-se à causa/demanda',
                'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão|dando-se)\s+(?:à|a)\s+(?:presente\s+)?(?:causa|demanda|ação)\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 5: "Valor da causa com por extenso"
            {
                'nome': 'valor da causa com por extenso',
                'regex': r'(?:valor|valor\s+da\s+causa|valor\s+da\s+demanda|valor\s+da\s+ação)\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})\s*\([A-Za-zÀ-ÿ\s]+\)',
                'confianca': 'alta'
            },
            
            # Padrão 6: "Fins fiscais/alçada antes"
            {
                'nome': 'fins fiscais/alçada antes',
                'regex': r'para\s+(?:os\s+)?fins\s+(?:fiscais|de\s+alçada|fiscais\s+e\s+de\s+alçada)[\.,]?\s*R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 7: "Dando-se à causa"
            {
                'nome': 'dando-se à causa',
                'regex': r'dando-se\s+(?:à|a)\s+causa\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 8: "Dar-se-á à causa"
            {
                'nome': 'dar-se-á à causa',
                'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão)\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 9: "Valor da causa explícito"
            {
                'nome': 'valor da causa explícito',
                'regex': r'valor\s+da\s+causa[:\s]+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 10: "Atribui-se à causa"
            {
                'nome': 'atribui-se à causa',
                'regex': r'atribui[r-]?se\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 11: "Seção valor da causa"
            {
                'nome': 'seção valor da causa',
                'regex': r'(?:DO|DA)\s+VALOR\s+DA\s+CAUSA.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # PADRÕES DE MÉDIA CONFIANÇA ==========================================
            
            {
                'nome': 'contexto causa-valor',
                'regex': r'(?:atribui|atribuir|atribuído|atribuindo).*?causa.*?valor.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'média'
            },
            
            {
                'nome': 'fins fiscais (contexto)',
                'regex': r'(?:para\s+fins|para\s+efeitos).*?(?:fiscais|meramente\s+fiscais).*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'média'
            },
        ]
        
        padrao_geral = r'R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})'
        todos_valores_brutos = re.findall(padrao_geral, pedido)
        
        todos_valores = []
        for valor in todos_valores_brutos:
            if '.' not in valor:
                numeros = re.sub(r'\D', '', valor)
                if len(numeros) >= 3:
                    parte_inteira = numeros[:-2]
                    centavos = numeros[-2:]
                    
                    if len(parte_inteira) > 3:
                        parte_inteira_formatada = ''
                        for i, digito in enumerate(reversed(parte_inteira)):
                            if i > 0 and i % 3 == 0:
                                parte_inteira_formatada = '.' + parte_inteira_formatada
                            parte_inteira_formatada = digito + parte_inteira_formatada
                        valor_formatado = f"{parte_inteira_formatada},{centavos}"
                    else:
                        valor_formatado = f"{parte_inteira},{centavos}"
                    
                    todos_valores.append(valor_formatado)
                else:
                    todos_valores.append(valor)
            else:
                todos_valores.append(valor)
        
        if todos_valores:
            for padrao in padroes:
                regex_modificado = padrao['regex'].replace(
                    r'([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                    r'([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})'
                )
                
                match = re.search(regex_modificado, pedido, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                if match:
                    valor_str = match.group(1)
                    
                    if '.' not in valor_str:
                        numeros = re.sub(r'\D', '', valor_str)
                        if len(numeros) >= 3:
                            parte_inteira = numeros[:-2]
                            centavos = numeros[-2:]
                            
                            if len(parte_inteira) > 3:
                                parte_inteira_formatada = ''
                                for i, digito in enumerate(reversed(parte_inteira)):
                                    if i > 0 and i % 3 == 0:
                                        parte_inteira_formatada = '.' + parte_inteira_formatada
                                    parte_inteira_formatada = digito + parte_inteira_formatada
                                valor_str = f"{parte_inteira_formatada},{centavos}"
                            else:
                                valor_str = f"{parte_inteira},{centavos}"
                    
                    confianca = padrao['confianca']
                    
                    try:
                        valor_numerico = float(valor_str.replace('.', '').replace(',', '.'))
                    except ValueError:
                        valor_numerico = None
                    
                    break
            
            if valor_numerico is None and todos_valores:
                valor_str = todos_valores[-1]
                confianca = 'baixa'
                
                try:
                    valor_numerico = float(valor_str.replace('.', '').replace(',', '.'))
                except ValueError:
                    valor_numerico = None
    
    # ========================================================================
    # EXTRAÇÃO DO CNPJ
    # ========================================================================
    
    cnpj = None
    
    if qualificacao and isinstance(qualificacao, str):
        padroes_cnpj = [
            # Padrão 1: Formato completo (aceita . ou , como separador, / ou \ como barra)
            # Ex: 60.746.948/0001-12 ou 60,746,948\0001-12
            r'(\d{2}[.,]\d{3}[.,]\d{3}[/\\]\d{4}[-\s]?\d{2})',
            
            # Padrão 2: Formato sem barra (com pontos ou vírgulas)
            # Ex: 60.746.948.0001-12 ou 60,746,948,0001-12
            r'(\d{2}[.,]\d{3}[.,]\d{3}[.,]\d{4}[-\s]?\d{2})',
            
            # Padrão 3: Com espaços ao redor dos separadores
            # Ex: 60.746.948 / 0001-12 ou 60,746,948 \ 0001 - 12
            r'(\d{2}\s*[.,]\s*\d{3}\s*[.,]\s*\d{3}\s*[/\\]\s*\d{4}\s*[-\s]?\s*\d{2})',
            
            # Padrão 4: Com espaços e ponto no lugar de barra
            # Ex: 60.746.948 . 0001-12
            r'(\d{2}\s*[.,]\s*\d{3}\s*[.,]\s*\d{3}\s*[.,]\s*\d{4}\s*[-\s]?\s*\d{2})',
            
            # Padrão 5: Formato flexível (qualquer combinação de separadores)
            # Ex: 60 746 948/0001-12 ou 60.746,948\0001 12
            r'(\d{2}[\s.,]*\d{3}[\s.,]*\d{3}[\s.,/\\]*\d{4}[\s\-]*\d{2})',
            
            # Padrão 6: CNPJ com "nº" ou "n°" ou "no" antes
            # Ex: CNPJ nº 60.746.948/0001-12
            r'(?:CNPJ|cnpj)[\s:]*(?:n[ºo°]?\.?[\s:]*)(\d{2}[.,\s]*\d{3}[.,\s]*\d{3}[/\\.,\s]*\d{4}[-\s]*\d{2})',
            
            # Padrão 7: Apenas 14 dígitos seguidos
            r'\b(\d{14})\b',
        ]
        
        for padrao_cnpj in padroes_cnpj:
            match = re.search(padrao_cnpj, qualificacao)
            if match:
                cnpj_bruto = match.group(1)
                cnpj_limpo = re.sub(r'\D', '', cnpj_bruto)
                
                if len(cnpj_limpo) == 14:
                    cnpj = cnpj_limpo
                    break
    
    return {
        'valor_numerico': valor_numerico,
        'confianca': confianca,
        'cnpj': cnpj
    }


In [7]:
# Aplicar a função analisa_regex em todo o dataframe
resultados = df.apply(lambda row: analisa_regex(row['ds_Pedidos'], row['ds_Qualificacao']), axis=1)

# Extrair os resultados para novas colunas
df['valor_numerico'] = resultados.apply(lambda x: x['valor_numerico'])
df['confianca_valor'] = resultados.apply(lambda x: x['confianca'])
df['cnpj'] = resultados.apply(lambda x: x['cnpj'])

print(df[['cd_causa', 'valor_numerico', 'confianca_valor', 'cnpj']].head(20))

      cd_causa  valor_numerico confianca_valor            cnpj
0   CIB0500064        21906.64            alta  07207996000150
1   CIB0505587        10000.00            alta  60746948000112
2   CIB0508201        11409.08            alta  60746948000112
3   CIB0514647         1256.04            alta  33055146000193
4   CIB0500604        15215.42            alta  60746948000112
5   CIB0511068        15000.00            alta            None
6   CIB0515030         5159.84            alta  60746948000112
7   CIB0515018             NaN            None  33055146004776
8   CIB0515702             NaN            None            None
9   CIB0514229        16804.00            alta  60746948000112
10  CIB0514230        16804.00            alta  60746948000112
11  CIB0524760        15059.94            alta  60746948000112
12  CIB0517405        12895.19            alta  60746948000112
13  CIB0540432        10000.00            alta  60746948000112
14  CIB0514066         5000.00            alta  6074694